## Random Forest with 5-fold CV, 6 repitition on 100k35F dataset
dataset: 35 features, 100k row subsampled dataset, 80:10:10 train:val:test split 

with optuna early stopping

20221011

In [1]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import optuna as opt
import warnings
warnings.filterwarnings("ignore")
import time
from sklearn.model_selection import cross_val_score, KFold
from sklearn.impute import SimpleImputer
from optuna.samplers import TPESampler

In [2]:
#import data
train_df = pd.read_parquet("100K35F_train_main.parquet.snappy")
val_df = pd.read_parquet("100K35F_val_main.parquet.snappy")
test_df = pd.read_parquet("100K35F_test_main.parquet.snappy")
test_df

,r_id,r_stars,r_stars_square,r_length,u_friends_count,u_review_count,u_month_age,u_comp_avg,u_n_elite_yrs,u_fans,...,r_stopwords/words,r_digit_cnt,r_noun_cnt,r_Adj_cnt,r_Adv_cnt,r_capital_word_cnt,r_quoted_word_cnt,r_hashtag_cnt,r_exclam_cnt,r_useful
0,1340676,1,1,305,1,1,37.547287,0.000000,0,0,...,0.42,1,59,17,27,12,0,0,0,1
1,941404,4,16,20,2,96,40.146040,0.135417,0,4,...,0.32,0,3,4,4,0,0,0,1,1
2,1312730,1,1,285,333,24,39.063471,0.083333,0,2,...,0.49,4,40,10,28,14,0,0,0,3
3,5155149,4,16,139,273,374,7.853903,2.077540,7,37,...,0.43,2,32,8,9,4,0,0,0,4
4,3706898,4,16,41,1,5,11.606999,0.000000,0,0,...,0.45,1,6,5,5,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4134229,5,25,95,1,12,34.229801,0.000000,0,0,...,0.48,0,15,8,8,0,0,0,0,1
9996,4059522,3,9,454,149,361,101.389702,0.883657,8,49,...,0.42,0,92,37,23,27,0,0,0,3
9997,5567999,3,9,347,352,286,92.625418,0.527972,5,36,...,0.46,2,71,26,28,10,0,0,3,2
9998,2980845,4,16,175,217,219,54.729315,1.205479,4,17,...,0.45,0,33,17,11,8,0,0,0,2


In [3]:
X_train, y_train = train_df.drop(['r_useful', 'r_id'], axis=1).values, train_df['r_useful'].values
X_val, y_val = val_df.drop(['r_useful', 'r_id'], axis=1).values, val_df['r_useful'].values
X_test, y_test = test_df.drop(['r_useful', 'r_id'], axis=1).values, test_df['r_useful'].values

In [ ]:
# Code reference: With many thx to Brian's XGBboost notebook: 100K32F_XGBoost.ipynb

In [4]:
N_OPTUNA_TRIALS = 50
K_FOLDS = 5 
N_REPS = 6
# early stopping threshold
T_ES = 20 
MODEL_PREFIX = "RF"

# Mean imputation and standardisation
model_pipe = Pipeline([
    ("imp", SimpleImputer()),
    ("ss", StandardScaler()),
    (MODEL_PREFIX, RandomForestRegressor())])

In [5]:
def hp_appender(hp_dict):
    '''Return dictionary where every key has the MODEL_PREFIX__ appended.'''
    new_dict = {}
    for key, val in hp_dict.items():
        new_dict[MODEL_PREFIX + "__" + key] = val
    return new_dict

In [6]:
def optuna_objective(trial, model, X, y):
    print(f"{time.strftime('%H:%M:%S', time.localtime())} | Running Optuna Trial: {trial.number}")
    
    
    hyperparams = {
        "n_estimators": trial.suggest_int('n_estimators', 1, 500, 25),
        "max_depth": trial.suggest_int('max_depth', 1, 500, 25),
        "min_samples_split": trial.suggest_int('min_samples_split', 2, 20),
        "max_features" : trial.suggest_categorical("max_features", ['sqrt', 'log2']),
        "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 20), 
        "random_state": 760, 
        "n_jobs": -1
    }
    
    print(f"with hyperparameters: {hyperparams}")
    hyperparams = hp_appender(hyperparams)
    model.set_params(**hyperparams)
    
    
    # Inner CV Loop
    avg_score = -cross_val_score(model, X, y,
        scoring="neg_root_mean_squared_error", cv=KFold(K_FOLDS)).mean()
    print(f"complete! average cv RMSE: {avg_score}")
    return avg_score

In [7]:
class early_stopping_check_callback:
    def __init__(self, threshold):
        self.threshold = threshold

    def __call__(self, study, trial):
        # stop study if the number of consecutive trials with no improvement is
        # at least the threshold.
        if trial.number - study.best_trial.number >= self.threshold:
            print("==== EARLY STOPPING ACTIVATED ====")
            study.stop()

In [8]:
def get_best_model(X, y):
    '''Obtains best hyperparameters using TPE sampling and cross-validation.'''
    study = opt.create_study(direction='minimize', sampler=TPESampler(seed=760))
    study.optimize(
        lambda trial: optuna_objective(trial, model_pipe, X, y),
        n_trials=N_OPTUNA_TRIALS,
        callbacks=[early_stopping_check_callback(T_ES)]) # early stopping
    return study.best_params

In [9]:
def fit_and_score(model, hps, X_train, y_train, X_test, y_test):
    model.set_params(**hp_appender(hps))
    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)

    # calculate scores
    rmse = mean_squared_error(y_test, y_preds, squared=False)
    mae = mean_absolute_error(y_test, y_preds)
    return rmse, mae

In [10]:
# actual repeated CV loop

repetition_results = {
    "hp": [],
    "rmse": [],
    "mae": []
}

for rep_i in range(N_REPS):
    print(f"====== Running repetition: {rep_i} ======")
    # shuffle training data. Make sure each iteration uses different seed
    cv_train = train_df.sample(frac=1, random_state=760+rep_i)
    # extract X and y arrays
    cv_X_train, cv_y_train = cv_train.drop(['r_useful', 'r_id'], axis=1).values, cv_train['r_useful'].values
    
    # obtain best hyperparameters via cross-validation and TPE sampling.
    best_params = get_best_model(cv_X_train, cv_y_train)
    
    rmse, mae = fit_and_score(
        model_pipe, best_params,
        cv_X_train, cv_y_train,
        X_val, y_val)

    # save results for this iteration
    repetition_results["rmse"].append(rmse)
    repetition_results["mae"].append(mae)
    repetition_results["hp"].append(best_params)

[I 2022-10-11 22:20:42,824] A new study created in memory with name: no-name-71da7892-4467-4541-bb79-45c972f763bd


====== Running repetition: 0 ======
22:20:42 | Running Optuna Trial: 0
with hyperparameters: {'n_estimators': 51, 'max_depth': 51, 'min_samples_split': 5, 'max_features': 'log2', 'min_samples_leaf': 18, 'random_state': 760, 'n_jobs': -1}


[I 2022-10-11 22:20:48,081] Trial 0 finished with value: 3.5844660070761094 and parameters: {'n_estimators': 51, 'max_depth': 51, 'min_samples_split': 5, 'max_features': 'log2', 'min_samples_leaf': 18}. Best is trial 0 with value: 3.5844660070761094.


complete! average cv RMSE: 3.5844660070761094
22:20:48 | Running Optuna Trial: 1
with hyperparameters: {'n_estimators': 76, 'max_depth': 451, 'min_samples_split': 14, 'max_features': 'sqrt', 'min_samples_leaf': 14, 'random_state': 760, 'n_jobs': -1}


[I 2022-10-11 22:20:52,809] Trial 1 finished with value: 3.5723312077282223 and parameters: {'n_estimators': 76, 'max_depth': 451, 'min_samples_split': 14, 'max_features': 'sqrt', 'min_samples_leaf': 14}. Best is trial 1 with value: 3.5723312077282223.


...

complete! average cv RMSE: 3.462754262743631
23:35:45 | Running Optuna Trial: 48
with hyperparameters: {'n_estimators': 276, 'max_depth': 76, 'min_samples_split': 5, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'random_state': 760, 'n_jobs': -1}


[I 2022-10-11 23:35:58,505] Trial 48 finished with value: 3.5105131389554125 and parameters: {'n_estimators': 276, 'max_depth': 76, 'min_samples_split': 5, 'max_features': 'sqrt', 'min_samples_leaf': 10}. Best is trial 40 with value: 3.452933018339876.


complete! average cv RMSE: 3.5105131389554125
23:35:58 | Running Optuna Trial: 49
with hyperparameters: {'n_estimators': 101, 'max_depth': 26, 'min_samples_split': 6, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'random_state': 760, 'n_jobs': -1}


[I 2022-10-11 23:36:04,699] Trial 49 finished with value: 3.4933728597555826 and parameters: {'n_estimators': 101, 'max_depth': 26, 'min_samples_split': 6, 'max_features': 'sqrt', 'min_samples_leaf': 5}. Best is trial 40 with value: 3.452933018339876.


complete! average cv RMSE: 3.4933728597555826


In [11]:
#final model performance on train+val
X_train_val = np.vstack((X_train, X_val))
y_train_val = np.hstack((y_train, y_val))
print(X_train_val.shape)
print(y_train_val.shape)

rmse = repetition_results['rmse']
best_best_i = np.argmin(rmse) # best = minimum RMSE
hp = repetition_results['hp']
best_best_hp = hp[best_best_i]

rmse, mae = fit_and_score(model_pipe, best_best_hp,
    X_train_val, y_train_val, X_test, y_test)

print(f"Best overall HP:{best_best_hp}")
print(f"Best overall RMSE: {rmse:.4}")
print(f"Best overall MAE: {mae:.4}")

(90000, 35)
(90000,)
Best overall HP:{'n_estimators': 376, 'max_depth': 201, 'min_samples_split': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1}
Best overall RMSE: 3.045
Best overall MAE: 1.452


Obtain mean and stdev statistics across the repetitions

In [16]:
print(repetition_results)

print("\nRMSE")
rmse = repetition_results['rmse']
print(f"mean: {np.mean(rmse):.4}, stdev: {np.std(rmse):.4}")

print("MAE")
mae = repetition_results['mae']
print(f"mean: {np.mean(mae):.4}, stdev: {np.std(mae):.4}")

{'hp': [{'n_estimators': 451, 'max_depth': 451, 'min_samples_split': 6, 'max_features': 'sqrt', 'min_samples_leaf': 1}, {'n_estimators': 376, 'max_depth': 201, 'min_samples_split': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1}, {'n_estimators': 301, 'max_depth': 201, 'min_samples_split': 6, 'max_features': 'log2', 'min_samples_leaf': 1}, {'n_estimators': 376, 'max_depth': 101, 'min_samples_split': 8, 'max_features': 'log2', 'min_samples_leaf': 1}, {'n_estimators': 301, 'max_depth': 201, 'min_samples_split': 6, 'max_features': 'log2', 'min_samples_leaf': 1}, {'n_estimators': 326, 'max_depth': 76, 'min_samples_split': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2}], 'rmse': [3.056026001750922, 3.0383367837412973, 3.0408979709172463, 3.045340648097417, 3.052460707132541, 3.0618777699985866], 'mae': [1.4885044849832474, 1.4890241261301798, 1.4854463588046913, 1.4863651813869234, 1.486576233917567, 1.4700901030661768]}

RMSE
mean: 3.049, stdev: 0.008364
MAE
mean: 1.484, stdev: 0.0064